## importing train data 

In [360]:
from pathlib import Path
from pprint import pprint
from collections import Counter
from typing import Dict, List
import json

from presidio_evaluator import InputSample
from presidio_evaluator.evaluation import Evaluator, ModelError
from presidio_evaluator.models import PresidioAnalyzerWrapper
from presidio_evaluator.experiment_tracking import get_experiment_tracker

import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [361]:
dataset_name = "data2.json"
dataset = InputSample.read_dataset_json(Path(Path.cwd().parent, "data modification", dataset_name))

print(len(dataset))

print (dataset[0])

tokenizing input: 100%|██████████| 2/2 [00:00<00:00,  7.57it/s]


2
Full text: 89200-3325 schools are next in line for education reform pilot program. Mobility team, prepare accordingly!
Spans: [Span(type: ZIPCODE, value: 89200-3325, char_span: [0: 10]), Span(type: JOBAREA, value: Mobility, char_span: [72: 80])]



In [362]:
def get_entity_counts(dataset: List[InputSample]) -> Dict:
    """Return a dictionary with counter per entity type."""
    entity_counter = Counter()
    for sample in dataset:
        for tag in sample.tags:
            entity_counter[tag] += 1
    return entity_counter


In [363]:
entity_counts = get_entity_counts(dataset)
print("Count per entity:")
pprint(entity_counts.most_common(), compact=True)

print("\nMin and max number of tokens in dataset: "\
f"Min: {min([len(sample.tokens) for sample in dataset])}, "\
f"Max: {max([len(sample.tokens) for sample in dataset])}")

print(f"Min and max sentence length in dataset: " \
f"Min: {min([len(sample.full_text) for sample in dataset])}, "\
f"Max: {max([len(sample.full_text) for sample in dataset])}")

print("\nExample InputSample:")
print(dataset[1])

Count per entity:
[('O', 55), ('ZIPCODE', 3), ('JOBAREA', 1), ('FIRSTNAME', 1), ('EMAIL', 1)]

Min and max number of tokens in dataset: Min: 20, Max: 41
Min and max sentence length in dataset: Min: 107, Max: 241

Example InputSample:
Full text: Jessyca, you should compare our performance to the industry averages. This includes leads, conversion rates, bounce rates, page views, average spend per customer, and customer acquisition costs. Send a report to Roosevelt_Kshlerin@yahoo.com.
Spans: [Span(type: FIRSTNAME, value: Jessyca, char_span: [0: 7]), Span(type: EMAIL, value: Roosevelt_Kshlerin@yahoo.com, char_span: [212: 240])]



In [364]:
print("A few examples sentences containing each entity:\n")
for entity in entity_counts.keys():
    samples = [sample for sample in dataset if entity in set(sample.tags)]
    if len(samples) > 1 and entity != "O":
        print(f"Entity: <{entity}> two example sentences:\n"
              f"\n1) {samples[0].full_text}"
              f"\n2) {samples[1].full_text}"
              f"\n------------------------------------\n")

A few examples sentences containing each entity:



In [365]:
from presidio_analyzer import AnalyzerEngine
# Loading the vanilla Analyzer Engine, with the default NER model.
analyzer_engine = AnalyzerEngine(default_score_threshold=0.4)

pprint(f"Supported entities for English:")
pprint(analyzer_engine.get_supported_entities("en"), compact=True)

print(f"\nLoaded recognizers for English:")
pprint([rec.name for rec in analyzer_engine.registry.get_recognizers("en", all_fields=True)], compact=True)

print(f"\nLoaded NER models:")
pprint(analyzer_engine.nlp_engine.models)

'Supported entities for English:'
['CREDIT_CARD', 'PERSON', 'IN_PASSPORT', 'DATE_TIME', 'IN_PAN', 'AU_ABN',
 'AU_MEDICARE', 'NRP', 'ORGANIZATION', 'PHONE_NUMBER', 'EMAIL_ADDRESS',
 'US_DRIVER_LICENSE', 'IP_ADDRESS', 'IBAN_CODE', 'URL', 'IN_AADHAAR',
 'LOCATION', 'US_ITIN', 'CRYPTO', 'SG_NRIC_FIN', 'IN_VEHICLE_REGISTRATION',
 'MEDICAL_LICENSE', 'AU_ACN', 'IN_VOTER', 'US_PASSPORT', 'UK_NHS',
 'US_BANK_NUMBER', 'AU_TFN', 'US_SSN']

Loaded recognizers for English:
['CreditCardRecognizer', 'UsBankRecognizer', 'UsLicenseRecognizer',
 'UsItinRecognizer', 'UsPassportRecognizer', 'UsSsnRecognizer', 'NhsRecognizer',
 'SgFinRecognizer', 'AuAbnRecognizer', 'AuAcnRecognizer', 'AuTfnRecognizer',
 'AuMedicareRecognizer', 'InPanRecognizer', 'InAadhaarRecognizer',
 'InVehicleRegistrationRecognizer', 'InPassportRecognizer', 'CryptoRecognizer',
 'DateRecognizer', 'EmailRecognizer', 'IbanRecognizer', 'IpRecognizer',
 'MedicalLicenseRecognizer', 'PhoneRecognizer', 'UrlRecognizer',
 'InVoterRecognizer', 'Sp

In [366]:

presidio_entities_map1 = dict(
  FIRSTNAME=  "PERSON",
  LASTNAME = "PERSON",
  MIDDLENAME="PERSON",
  PERSON = "PERSON",

  DATE="DATE_TIME",
  TIME="DATE_TIME",
  DOB="DATE_TIME" ,
  DATE_TIME = "DATE_TIME",

  EMAIL="EMAIL_ADDRESS",
  EMAIL_ADDRESS="EMAIL_ADDRESS",

  PREFIX="TITLE",
  TITLE = "TITLE",

  URL="URL",

  STREET="LOCATION",
  STATE="LOCATION" , 
  CITY="LOCATION" , 
  COUNTY="LOCATION",
  SECONDARYADDRESS="LOCATION" ,
  LOCATION = "LOCATION",

  PHONEIMEI="PHONE_NUMBER",
  PHONENUMBER="PHONE_NUMBER",
  PHONE_NUMBER = "PHONE_NUMBER",

  IPV4="IP_ADDRESS",
  IPV6="IP_ADDRESS",
  IP="IP_ADDRESS",
  IP_ADDRESS = "IP_ADDRESS",

  CREDITCARDNUMBER="CREDIT_CARD",
  CREDIT_CARD = "CREDIT_CARD",

  ZIPCODE="ZIP_CODE",
  ZIP_CODE ="ZIP_CODE",

  COMPANYNAME="ORGANIZATION",
  ORGANIZATION= "ORGANIZATION",

  IBAN="IBAN_CODE",
  IBAN_CODE = "IBAN_CODE",

  SSN="US_SSN",
  US_SSN = "US_SSN",

  AGE="AGE",




  AMOUNT="O",
  USERNAME="O",
  JOBTITLE="O",
  JOBAREA="O",
  ACCOUNTNAME="O",
  ACCOUNTNUMBER="O",
  JOBTYPE="O",
  BUILDINGNUMBER="O" ,
  CURRENCYSYMBOL="O" ,
  PASSWORD="O",
  SEX="O",
  GENDER="O",
  BITCOINADDRESS="O",
  MASKEDNUMBER="O",
  USERAGENT="O",
  CURRENCY="O",
  ETHEREUMADDRESS="O",
  NEARBYGPSCOORDINATE="O",
  CREDITCARDISSUER="O",
  ORDINALDIRECTION="O",
  MAC="O" ,
  VEHICLEVRM="O",
  EYECOLOR="O",
  CREDITCARDCVV="O",
  HEIGHT="O" ,
  LITECOINADDRESS="O",
  VEHICLEVIN="O" ,
  CURRENCYCODE="O",
  CURRENCYNAME="O" ,
  BIC="O",
  PIN="O",
  O= "O",

)







In [367]:
#entities_mapping=PresidioAnalyzerWrapper.presidio_entities_map 
entities_mapping = presidio_entities_map1
print("Using this mapping between the dataset and Presidio's entities:")
pprint(entities_mapping, compact=True)


dataset = Evaluator.align_entity_types(
    dataset, 
    entities_mapping=entities_mapping, 
    allow_missing_mappings=True
)
new_entity_counts = get_entity_counts(dataset)
print("\nCount per entity after alignment:")
pprint(new_entity_counts.most_common(), compact=True)

dataset_entities = list(new_entity_counts.keys())


Using this mapping between the dataset and Presidio's entities:
{'ACCOUNTNAME': 'O',
 'ACCOUNTNUMBER': 'O',
 'AGE': 'AGE',
 'AMOUNT': 'O',
 'BIC': 'O',
 'BITCOINADDRESS': 'O',
 'BUILDINGNUMBER': 'O',
 'CITY': 'LOCATION',
 'COMPANYNAME': 'ORGANIZATION',
 'COUNTY': 'LOCATION',
 'CREDITCARDCVV': 'O',
 'CREDITCARDISSUER': 'O',
 'CREDITCARDNUMBER': 'CREDIT_CARD',
 'CREDIT_CARD': 'CREDIT_CARD',
 'CURRENCY': 'O',
 'CURRENCYCODE': 'O',
 'CURRENCYNAME': 'O',
 'CURRENCYSYMBOL': 'O',
 'DATE': 'DATE_TIME',
 'DATE_TIME': 'DATE_TIME',
 'DOB': 'DATE_TIME',
 'EMAIL': 'EMAIL_ADDRESS',
 'EMAIL_ADDRESS': 'EMAIL_ADDRESS',
 'ETHEREUMADDRESS': 'O',
 'EYECOLOR': 'O',
 'FIRSTNAME': 'PERSON',
 'GENDER': 'O',
 'HEIGHT': 'O',
 'IBAN': 'IBAN_CODE',
 'IBAN_CODE': 'IBAN_CODE',
 'IP': 'IP_ADDRESS',
 'IPV4': 'IP_ADDRESS',
 'IPV6': 'IP_ADDRESS',
 'IP_ADDRESS': 'IP_ADDRESS',
 'JOBAREA': 'O',
 'JOBTITLE': 'O',
 'JOBTYPE': 'O',
 'LASTNAME': 'PERSON',
 'LITECOINADDRESS': 'O',
 'LOCATION': 'LOCATION',
 'MAC': 'O',
 'MASKED

In [368]:
print (dataset[0])

Full text: 89200-3325 schools are next in line for education reform pilot program. Mobility team, prepare accordingly!
Spans: [Span(type: ZIP_CODE, value: 89200-3325, char_span: [0: 10]), Span(type: O, value: Mobility, char_span: [72: 80])]



In [369]:
# Set up the experiment tracker to log the experiment for reproducibility
experiment = get_experiment_tracker()
 
# Create a wrapper for Presidio to be used within the presidio-evaluator framework
model = PresidioAnalyzerWrapper(analyzer_engine, 
                                entity_mapping=entities_mapping)

# Create the evaluator object
evaluator = Evaluator(model=model)


# Track model and dataset params
params = {"dataset_name": dataset_name, "model_name": model.name}
params.update(model.to_log())
experiment.log_parameters(params)
experiment.log_dataset_hash(dataset)
experiment.log_parameter("entity_mappings", json.dumps(entities_mapping))

--------
Entities supported by this Presidio Analyzer instance:
CREDIT_CARD, PERSON, IN_PASSPORT, DATE_TIME, IN_PAN, AU_ABN, AU_MEDICARE, NRP, ORGANIZATION, PHONE_NUMBER, EMAIL_ADDRESS, US_DRIVER_LICENSE, IP_ADDRESS, IBAN_CODE, URL, IN_AADHAAR, LOCATION, US_ITIN, CRYPTO, SG_NRIC_FIN, IN_VEHICLE_REGISTRATION, MEDICAL_LICENSE, AU_ACN, IN_VOTER, US_PASSPORT, UK_NHS, US_BANK_NUMBER, AU_TFN, US_SSN


In [370]:
## Run experiment

evaluation_results = evaluator.evaluate_all(dataset)
results = evaluator.calculate_score(evaluation_results)

# Track experiment results
experiment.log_metrics(results.to_log())
entities, confmatrix = results.to_confusion_matrix()
experiment.log_confusion_matrix(matrix=confmatrix, 
                                labels=entities)

# Plot output
plotter = evaluator.Plotter(model=model, 
                            results=results, 
                            output_folder = ".", 
                            model_name = model.name, 
                            beta = 2)


# end experiment
experiment.end()

Mapping entity values using this dictionary: {'FIRSTNAME': 'PERSON', 'LASTNAME': 'PERSON', 'MIDDLENAME': 'PERSON', 'PERSON': 'PERSON', 'DATE': 'DATE_TIME', 'TIME': 'DATE_TIME', 'DOB': 'DATE_TIME', 'DATE_TIME': 'DATE_TIME', 'EMAIL': 'EMAIL_ADDRESS', 'EMAIL_ADDRESS': 'EMAIL_ADDRESS', 'PREFIX': 'TITLE', 'TITLE': 'TITLE', 'URL': 'URL', 'STREET': 'LOCATION', 'STATE': 'LOCATION', 'CITY': 'LOCATION', 'COUNTY': 'LOCATION', 'SECONDARYADDRESS': 'LOCATION', 'LOCATION': 'LOCATION', 'PHONEIMEI': 'PHONE_NUMBER', 'PHONENUMBER': 'PHONE_NUMBER', 'PHONE_NUMBER': 'PHONE_NUMBER', 'IPV4': 'IP_ADDRESS', 'IPV6': 'IP_ADDRESS', 'IP': 'IP_ADDRESS', 'IP_ADDRESS': 'IP_ADDRESS', 'CREDITCARDNUMBER': 'CREDIT_CARD', 'CREDIT_CARD': 'CREDIT_CARD', 'ZIPCODE': 'ZIP_CODE', 'COMPANYNAME': 'ORGANIZATION', 'ORGANIZATION': 'ORGANIZATION', 'IBAN': 'IBAN_CODE', 'IBAN_CODE': 'IBAN_CODE', 'SSN': 'US_SSN', 'US_SSN': 'US_SSN', 'AGE': 'AGE', 'AMOUNT': 'O', 'USERNAME': 'O', 'JOBTITLE': 'O', 'JOBAREA': 'O', 'ACCOUNTNAME': 'O', 'ACCOUN

In [371]:

print (dataset[0])

Full text: 89200-3325 schools are next in line for education reform pilot program. Mobility team, prepare accordingly!
Spans: []



In [372]:
plotter.plot_scores()

In [ ]:
plotter.plot_confusion_matrix(entities=entities, confmatrix=confmatrix)

In [359]:
plotter.plot_most_common_tokens()